# PDMP sampling of conditioned diffusions

This notebook is a simple adaptation of the ou.jl file from https://github.com/SebaGraz/ZZDiffusionBridge/tree/master/scripts/examples 

It is to be ran in the same location as ou.jl within the above repository

The viewer is encouraged to look at https://github.com/mschauer/ZigZagBoomerang.jl for further content on using PDMP samplers for function space sampling. 

In [1]:
using Pkg;
Pkg.add("Tables")
Pkg.add("ProgressMeter")
Pkg.add("CSV")
Pkg.add("NPZ")

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [2]:
include("../../src/ZZDiffusionBridge.jl")
using LinearAlgebra
"""
    OUSDE <: AbstractModel

dX_t = ν(μ - X_t)dt + dB_t
ν := intensity
μ := mean reversion
M::Array{Float64, 2} := matrix whose element i,j is equal to ∫_0^T ϕ_i ϕ_j dt
V::Vector{Float64} :=  vector whose element i is equal to ∫_0^T ϕ_i dt
bound1::Vector{Float64} :=  vector whose element i is equal to ∫_0^T upbar{ϕ}_i ϕ_j dt
bound2::Vector{Float64} :=  vector whose element i is equal to ∫_0^T downbar{ϕ}_i ϕ_j dt
"""

struct OUSDE <: AbstractModel
    μ::Float64
    ν::Float64
    M::Array{Float64, 2}
    V::Vector{Float64}
    bound1::Vector{Float64}
    bound2::Vector{Float64}
    function OUSDE(μ, ν, L, T)
        new(μ, ν, generate_matrix(L, T), generate_vector(L, T), generate_bound1(L,T), generate_bound2(L,T))
    end
end

dependence_strucute(::OUSDE) = PartialIndependence()
sampling_scheme(::OUSDE) = Regular()

sampling_scheme (generic function with 1 method)

In [3]:
#Poisson rates
"""
    wait_gengaus(a,b,u)
obtaining waiting time for Inhomogeneous Poisson Process
with rate of the form λ(t) = (a + b*t)^+, `a`,`b` ∈ R, `u` random variable
"""
function wait_gengaus(a,b,u)
    if b > 0
        if a < 0
            τ = sqrt(-log(u)*2.0/b) - a/b
        else #a[i]>0
            τ = sqrt((a/b)^2 - log(u)*2.0/b) - a/b
        end
    elseif  b == 0
        if a > 0
            τ = -log(u)/a
        else #a[i] <= 0
            τ = Inf
        end
    else #b[i] < 0
        if a <= 0
            τ = Inf
        elseif -log(u) <= -a^2/b + a^2/(2*b)
            τ = - sqrt((a/b)^2 - log(u)*2.0/b) - a/b
        else
            τ = Inf
        end
    end
end

"""
    λbar(n, S::System, X::OUSDE , u, v)

Poisson time (real bound) for the coefficient `n`
of model `OUSDE` starting at `u` and ending at `v`

λbar = λ No upper bound and no accept reject step
"""
function λbar(n, S::System, X::OUSDE , u::Float64, v::Float64, t::Float64)
    a = S.θ[n]*(X.ν*X.ν*(dot(X.M[n,:], S.ξ) + X.bound1[n]*v + X.bound2[n]*u - X.μ*X.V[n]) + S.ξ[n])
    b = S.θ[n]*((dot(X.M[:,n], S.θ))*X.ν*X.ν + S.θ[n])
    return wait_gengaus(a, b, rand())
end


λbar

In [4]:
function run_PDMP(alpha::Float64 = 0.7,T::Float64 = 50.0, clock::Float64 = 10000.0, L::Int64 = 6,u::Float64 = - Float64(π),v::Float64 = 3*Float64(π),prog_bar::Bool = false)
    X = SinSDE(alpha, L, T)
    XX = zz_sampler(X, T, L, u, v, clock,prog_bar)
    return XX
end

run_PDMP (generic function with 8 methods)

In [5]:
function runall(SHORT = false)
    Random.seed!(1)
    T = 10.0
    clock = 1000.0
    L = 6
    ν = 1.0
    μ = -5
    u = -1.0
    v = 2.0
    X = OUSDE(μ, ν, L, T)
    XX = zz_sampler(X, T, L, u, v, clock)
    if SHORT == false
        burning = 10.0    #burning
        f = clock - 1.0; n = 100
        db = (f-burning)/n
        b =  burning:db:f
        p = plotmixing(XX, b, T, L, u, v)
        hline!(p, [-5.0], color = :blue)
        xaxis!(p, "t")
        yaxis!(p, "X_t")
        display(p)
        #plot the mean of the process
    end
    return XX
end

runall (generic function with 2 methods)

In [6]:
function run_OU_PDMP(ν::Float64 = 1.0,μ::Float64 = -5.0,T::Float64 = 10.0, clock::Float64 = 1000.0, L::Int64 = 6,u::Float64 = -1.0,v::Float64 = 2.0,prog_bar::Bool = false)
    X = OUSDE(μ, ν, L, T)
    XX = zz_sampler(X, T, L, u, v, clock,prog_bar)
    return XX
end

run_OU_PDMP (generic function with 9 methods)

In [7]:
function extract_values(y::Array{Skeleton,1},b,L::Int64,T::Float64, u::Float64, v::Float64, trasform = x -> x)
    P = []
    
#     @showprogress for i in b
    for i in b
        ξ_interp = FindCoordinates(y, i).ξ
        dx = fs_expansion(ξ_interp, u, v, L, T)
        push!(P, trasform.(dx))
#         print("Extracting sample $(i) \r")
#         flush(stdout)
    end
    return P
end

extract_values (generic function with 2 methods)

In [11]:
# Interval length for the functions
T = 4.0
# Final clock time for the sampler
clock = 20_000.0
# Dictates number of basis elements to use in sampler
L = 6
# Parameter in the SDE
ν = 1.0
μ = -5.0
# Start and end points of the diffusion
u = -1.0
v = 2.0
# burn in period
burning = clock/10.0
# Reduce clock size by one to avoid overflow issues
f = clock - 1.0
# Number of samples to extract
n = 10_000
# Form extraction grid
db = (f-burning)/n
b =  burning:db:f
# Form sample path discretisation grid
dt = range(0.0, stop=T, length=2<<(L) + 1);

In [12]:
prog_bar = true
# Run the sampler
E = run_OU_PDMP(ν,μ,T,clock,L,u,v,prog_bar)
# Extract the samples
R = extract_values(E,b,L,T,u,v)
# Wrangle output into form that can be stored, the 2<<(L)+1 is due
# to how many points the sampled functions are evaluated at. It is due
# to the choice of basis in the sampler.
S = reshape(collect(Iterators.flatten(R)),(2<<(L)+1,n + 1));

Progress: 100%|█████████████████████████████████████████| Time: 0:00:12


In [10]:
using NPZ
npzwrite("OU_paths/OU_PDMP_Sample_Quality_Data_T_4.npy", S)